# Generate electrode coordinates for Berke Lab probes

## Get electrode coordinates for custom Berke Lab silicon probes


In [6]:
# Set up specs for all Berke Lab custom silicon probes

## 256-ch Silicon Probe, 3mm length, 66um pitch
# 32 shanks, 8 electrodes per shank. Each shank is 3mm long.
# Shanks are 66um apart. There is 30um between electrodes on each shank.
# Odd shanks are vertically offset by 1/2 the electrode pitch (15um).
# Shanks are numbered "left to right": 1 (leftmost shank) to 32 (rightmost shank).
# Electrodes are numbered "top to bottom": 1 (top, most dorsal) to 8 (tip, most ventral) on each shank.
probe_256ch_3mm_66um = {
    'num_shanks': 32,
    'electrodes_per_shank': 8,
    'shank_pitch': 66,          # um between shanks
    'electrode_pitch': 30,      # um vertically between electrodes
    'offset_odd_shanks': True,
    'top_to_bottom': True
}

## 256-ch Silicon Probe, 6mm length, 80um pitch
# 32 shanks, 8 electrodes per shank. Each shank is 6mm long.
# Shanks are 80um apart. There is 30um between electrodes on each shank.
# Odd shanks are vertically offset by 1/2 the electrode pitch (15um).
# Shanks are numbered "left to right": 1 (leftmost shank) to 32 (rightmost shank).
# Electrodes are numbered "top to bottom": 1 (top, most dorsal) to 8 (tip, most ventral) on each shank.
probe_256ch_6mm_80um = {
    'num_shanks': 32,
    'electrodes_per_shank': 8,
    'shank_pitch': 80,          # um between shanks
    'electrode_pitch': 30,      # um vertically between electrodes
    'offset_odd_shanks': True,
    'top_to_bottom': True
}

# --------------------------------------------------------------------------------------------------
# IMPORTANT NOTE / TODO: Dan says that for the 256ch probes we start with more ventral tip on the left.
# This is the opposite of what I currently have here. My current coordinates are based on what was used for 
# processing Tim's data and match what Dan's matlab code does, so I am not sure what is correct.
# This shouldn't effect spike sorting output, but really needs to be confirmed for correctness. 
# --------------------------------------------------------------------------------------------------

## 252-ch Silicon Probe, 4mm length, 80um pitch
# 21 shanks, 12 electrodes per shank. Each shank is 4mm long.
# Shanks are 80um apart. There is 25um between electrodes on each shank.
# Odd shanks are vertically offset by 1/2 the electrode pitch (12.5um).
# Shanks are numbered "left to right": 1 (leftmost shank) to 21 (rightmost shank).
# Electrodes are numbered "bottom to top": 1 (tip, most ventral) to 12 (top, most dorsal) on each shank.

probe_252ch_4mm_80um = {
    'num_shanks': 21,
    'electrodes_per_shank': 12,
    'shank_pitch': 80,          # um between shanks
    'electrode_pitch': 25,      # um vertically between electrodes
    'offset_odd_shanks': True,
    'top_to_bottom': False
}

## 252-ch Silicon Probe, 10mm length, 100um pitch
# 21 shanks, 12 electrodes per shank. Each shank is 10mm long.
# Shanks are 100um apart. There is 25um between electrodes on each shank.
# Odd shanks are vertically offset by 1/2 the electrode pitch (12.5um).
# Shanks are numbered "left to right": 1 (leftmost shank) to 21 (rightmost shank).
# Electrodes are numbered "bottom to top": 1 (tip, most ventral) to 12 (top, most dorsal) on each shank.
probe_252ch_10mm_100um = {
    'num_shanks': 21,
    'electrodes_per_shank': 12,
    'shank_pitch': 100,         # um between shanks
    'electrode_pitch': 25,      # um vertically between electrodes
    'offset_odd_shanks': True,
    'top_to_bottom': False
}

# Dict of all Berke Lab custom probes
probes = {
    '256-ch Silicon Probe, 3mm length, 66um pitch': probe_256ch_3mm_66um,
    '256-ch Silicon Probe, 6mm length, 80um pitch': probe_256ch_6mm_80um,
    '252-ch Silicon Probe, 4mm length, 80um pitch': probe_252ch_4mm_80um,
    '252-ch Silicon Probe, 10mm length, 100um pitch': probe_252ch_10mm_100um
}


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

save_files = True

def generate_electrode_coords(num_shanks, electrodes_per_shank, shank_pitch, electrode_pitch,
                               offset_odd_shanks=True, top_to_bottom=True):
    coords = []
    # Get x coordinate based on shank
    for shank in range(1, num_shanks + 1):  # 1-based shank
        x = shank * shank_pitch
        is_odd_shank = shank % 2
        vertical_offset = (electrode_pitch / 2) if (offset_odd_shanks and is_odd_shank) else 0
        # Get y coordinate based on electrode (+ y offset every other shank)
        for e in range(1, electrodes_per_shank + 1): # 1-based electrode
            y_idx = (electrodes_per_shank + 1 - e) if top_to_bottom else e
            y = y_idx * electrode_pitch + vertical_offset
            coords.append({
            "shank": shank,
            "electrode": e,
            "x_um": x,
            "y_um": y
        })
    return coords
    
def plot_electrode_coords(coords, title):
    x, y, labels = [], [], []

    for coord in coords:
        x.append(coord['x_um'])
        y.append(coord['y_um'])
        labels.append(f"S{coord['shank']}E{coord['electrode']}")

    fig, ax = plt.subplots(figsize=(16, 6))
    ax.scatter(x, y, c='blue', s=10)

    # Add labels for each electrode
    for i, label in enumerate(labels):
        ax.text(x[i], y[i], label, fontsize=8, ha='center', va='bottom', color='black')

    ax.set_title(title)
    ax.set_xlim(min(x) - 100, max(x) + 100)
    ax.set_ylim(min(y) - 100, max(y) + 100)
    ax.set_xlabel("X (µm)")
    ax.set_ylabel("Y (µm)")
    plt.show()
    return fig

# Iterate through the probes and generate coordinates and plot for each
for probe_name, probe_data in probes.items():
    coords = generate_electrode_coords(**probe_data)
    fig = plot_electrode_coords(coords, title=probe_name)
    coords_df = pd.DataFrame(coords)
    display(coords_df)
    if save_files:
        save_name = f"{probe_name.lower().replace(' ', '_').replace(',', '').replace('-','')}"
        save_name = save_name.replace('silicon_', '')
        coords_df.to_csv(f"{save_name}_coords.csv", index=False)
        fig.savefig(f"{save_name}_coords.png", dpi=300, bbox_inches="tight")
        plt.close()


## Get coordinates for all Neuropixels 2.0 (multishank) electrodes
4 shanks x 1280 electrodes per shank for 5120 total recording sites.

Shanks are 250um apart. There are 2 electrodes per row on each shank.

There is 15 um vertically and 32 um horizontally between electrodes on each shank.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

electrodes = []

# Specs for Neuropixels 2.0 multishank probe
num_shanks = 4
electrodes_per_shank = 1280
inter_shank_spacing = 250  # um between shanks
horizontal_spacing = 32    # um between columns
vertical_spacing = 15      # um between rows
x_offset = 8               # x coordinates start at 8 instead of 0 

for shank in range(num_shanks):
    base_index = shank * electrodes_per_shank
    for site_in_shank in range(electrodes_per_shank):
        electrode = base_index + site_in_shank
        column = site_in_shank % 2  # 0 (left), 1 (right)
        row = site_in_shank // 2

        x = shank * inter_shank_spacing + column * horizontal_spacing + x_offset
        y = row * vertical_spacing

        electrodes.append({
            "electrode": electrode,
            "shank": shank,
            "shank_column": column,
            "shank_row": row,
            "x_um": x,
            "y_um": y
        })

# Save to a csv
df = pd.DataFrame(electrodes)
#df.to_csv("neuropixels_2.0_multishank_electrode_coords.csv", index=False)

# Plot it
plt.figure(figsize=(10, 25))
plt.scatter(df['x_um'], df['y_um'], s=1, color='black')
plt.title("Neuropixels 2.0 (multishank) Electrode Layout")
plt.xlabel("X (µm)")
plt.ylabel("Y (µm)")
plt.xlim(df['x_um'].min() - 50, df['x_um'].max() + 50)
#plt.savefig("neuropixels_2.0_multishank_all_electrodes.png", dpi=300, bbox_inches='tight')
plt.show()
